In [1]:
import os
import json

def getFpDict(fileName):
    fpDict = {}
    with open(fileName, "r+") as fp:
        while not (fp.tell() == os.fstat(fp.fileno()).st_size):
            fpPos = fp.tell()
            line = fp.readline().split('-')
            fpDict[line[0]] = fpPos
    return fpDict

def getTermInfo(term:str, fpDict:dict, fileName):
    termInfo = None
    with open(fileName, "r+") as fp:
        fp.seek(fpDict[term])
        termInfo = fp.readline().strip().split('-')[1]
    return json.loads(termInfo)

In [3]:
fpDict1 = getFpDict('Data/Posting1.txt')
fpDict2 = getFpDict('Data/Posting2.txt')

In [5]:
term = 'uci'
t1 = getTermInfo(term, fpDict1, 'Data/Posting1.txt')
t2 = getTermInfo(term, fpDict2, 'Data/Posting2.txt')

print(t1)
print(t2)

{'df': 772, 'tf': {'1': 6, '2': 6, '3': 6, '4': 3, '5': 2, '6': 2, '7': 3, '8': 2, '9': 2, '10': 3, '11': 1, '12': 1, '13': 1, '14': 1, '15': 1, '16': 1, '17': 3, '18': 1, '19': 1, '20': 1, '21': 1, '22': 1, '23': 1, '24': 1, '25': 1, '26': 1, '27': 1, '28': 1, '29': 1, '30': 1, '31': 1, '32': 1, '33': 4, '34': 7, '35': 1, '36': 1, '37': 1, '38': 3, '39': 1, '40': 2, '41': 1, '42': 1, '43': 1, '44': 1, '45': 1, '46': 1, '47': 1, '48': 4, '49': 1, '50': 1, '51': 1, '52': 1, '53': 1, '54': 1, '55': 1, '56': 1, '57': 1, '58': 1, '59': 1, '60': 1, '61': 1, '62': 2, '63': 1, '64': 1, '65': 2, '66': 6, '67': 1, '68': 1, '69': 1, '70': 1, '71': 3, '72': 1, '73': 1, '74': 1, '75': 3, '76': 1, '77': 1, '78': 1, '79': 1, '80': 1, '81': 1, '82': 1, '83': 1, '84': 1, '85': 1, '86': 1, '87': 1, '88': 3, '89': 1, '90': 1, '91': 1, '92': 2, '93': 1, '94': 1, '95': 1, '96': 1, '97': 5, '98': 1, '99': 13, '100': 1, '101': 1, '102': 1, '103': 1, '104': 2, '105': 1, '106': 1, '107': 1, '109': 1, '110': 1

In [8]:
#Merge
t1['df'] += t2['df']
t1['tf'].update(t2['tf'])
t1['docIds'].extend(t2['docIds'])

print(t1)

{'df': 1309, 'tf': {'1': 6, '2': 6, '3': 6, '4': 3, '5': 2, '6': 2, '7': 3, '8': 2, '9': 2, '10': 3, '11': 1, '12': 1, '13': 1, '14': 1, '15': 1, '16': 1, '17': 3, '18': 1, '19': 1, '20': 1, '21': 1, '22': 1, '23': 1, '24': 1, '25': 1, '26': 1, '27': 1, '28': 1, '29': 1, '30': 1, '31': 1, '32': 1, '33': 4, '34': 7, '35': 1, '36': 1, '37': 1, '38': 3, '39': 1, '40': 2, '41': 1, '42': 1, '43': 1, '44': 1, '45': 1, '46': 1, '47': 1, '48': 4, '49': 1, '50': 1, '51': 1, '52': 1, '53': 1, '54': 1, '55': 1, '56': 1, '57': 1, '58': 1, '59': 1, '60': 1, '61': 1, '62': 2, '63': 1, '64': 1, '65': 2, '66': 6, '67': 1, '68': 1, '69': 1, '70': 1, '71': 3, '72': 1, '73': 1, '74': 1, '75': 3, '76': 1, '77': 1, '78': 1, '79': 1, '80': 1, '81': 1, '82': 1, '83': 1, '84': 1, '85': 1, '86': 1, '87': 1, '88': 3, '89': 1, '90': 1, '91': 1, '92': 2, '93': 1, '94': 1, '95': 1, '96': 1, '97': 5, '98': 1, '99': 13, '100': 1, '101': 1, '102': 1, '103': 1, '104': 2, '105': 1, '106': 1, '107': 1, '109': 1, '110': 